# **Промежуточная аттестация по курсу (Python)**

## **ЗАДАНИЕ**

В ячейке ниже представлен код, генерирующий DataFrame, который состоит всего из 1 столбца. Задача состоит в том, чтобы перевести его в one hot вид. Нельзя использовать get_dummies.





---





---



In [55]:
# Инсталляция эмулятора терминала, который можно запустить в Colab
# (ttps://github.com/singhsidhukuldeep/Google-Colab-Shell):

!pip install google-colab-shell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=5bc882e922591bb83c60091dd10e912be2b4b05d31432e82b3cbb66661a15f02
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


In [56]:
from google_colab_shell import getshell

In [57]:
# Запуск эмулятора терминала (для работы с git и GitHub):

getshell()



---



In [58]:
import pandas as pd

import random

In [155]:
lst = ['robot'] * 10
lst += ['human'] * 10
random.shuffle(lst)
data = pd.DataFrame({'whoAmI':lst})

data.head()

,whoAmI
0,human
1,robot
2,human
3,human
4,robot




---



## **РЕШЕНИЕ:**

### ВАРИАНТ

In [161]:
# Трансфоммируем единственное поле с данными в ещё одно поле индекса:

one_hot_df = data.set_index(keys = 'whoAmI', append = True)

one_hot_df.head()

,
,whoAmI
0,human
1,robot
2,human
3,human
4,robot


In [162]:
# Добавляем вспомогателный столбец для заполнения значений:

one_hot_df['code'] = 1

one_hot_df.head()

,,code
,whoAmI,
0,human,1
1,robot,1
2,human,1
3,human,1
4,robot,1


In [163]:
# Производим "сворачивание" датафрейма при помощи метода unstack()
# (при этом отстутствующие значения заполняются нулями):

one_hot_df = one_hot_df.unstack(fill_value = 0)

one_hot_df.head()

code      
whoAmI human robot
0          1     0
1          0     1
2          1     0
3          1     0
4          0     1

In [164]:
# Убираем верхний уровень в "шапке" датафрейма
# (т.е. удаляем уровень из мультииндекса):

one_hot_df.columns = one_hot_df.columns.droplevel()
one_hot_df.index.rename('')

one_hot_df.head()

whoAmI,human,robot
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [170]:
print(one_hot_df.index.name)

None




---





---

